In [1]:
# This notebook simulates the Cord system under PI (proportional integral) control in
# a Monte Carlo setting to assess average control energy per variable. 
# Integration is used via a 4th-order Runge Kutta algothm.
# The dynamical equations (including of the controllers) are implemented in
# dvCord which can be readily adapted for other systems or other control laws, which can
# be computed outside dvCord and passed as a parameter to the function. 
#
# For details on the method and analysis of results, please consult
# Aguirre, L. A., Freitas, F. B. and Letellier, C., ``Numerical interpretation of 
# controllability coefficients in nonlinear dynamics'', Communications in Nonlinear 
# Science and Numerical Simulation, vol. 116:106875, 2023. DOI: 10.1016/j.cnsns.2022.106875.
#
# LAA BH 02/11/21

import numpy as np
import matplotlib.pyplot as plt


In [2]:
def dvCord(x, e, ep, Ki, Kp, t, a, b, F, G):
#
# implements the vector field for the Cord system with PI control.
# here the controls are state variables, so they come in the first 3 positions of x
# x state vector
# e = [ex ey ez] control error and its derivative ep
# Ki and Kp are the vectors of PI parameters. 
# xd time derivative of x (vector field at x)

# First published in
#
# Aguirre, L.A., Letellier, C., Investigating observability properties from data in nonlinear dynamics, 
# Phisical Reivew E, 83:066209, 2011. DOI: 10.1103/PhysRevE.83.066209.
#
# Analysed in
#
# Letellier, C., Aguirre, L.A., Required criteria for recognizing new types of chaos: Application to the cord attractor, 
# Phisical Reivew E, 85:036204, 2012. DOI: 10.1103/PhysRevE.85.036204.

    xd = np.zeros((6,1));

    # Differential equations of three PI controllers
    xd[0] = Ki[0]*e[0] + Kp[0]*ep[0]
    xd[1] = Ki[1]*e[1] + Kp[1]*ep[1]
    xd[2] = Ki[2]*e[2] + Kp[2]*ep[2]
    
    
    # Differential equations of the Cord vector field with PI control action
    xd[3] = -x[4] - x[5] -a*x[3] + a*F + x[0]
    xd[4] = x[3]*x[4] - b*x[3]*x[5] - x[4] + G + x[1]
    xd[5] = b*x[3]*x[4] + x[3]*x[5] -x[5] + x[2]

    return xd


In [3]:
def rkCord(x0, e, ep, Ki, Kp, h, t, a, b, F, G):
# 
# implements 4th-order Runge Kutta numerical integration algorithm 
# x0 is the state vector (before calling the integration function - it is 
# the initial condition of the current step)
# e = [ex ey ez] control error and its derivative ep
# Ki and Kp are the vectors of PI parameters. 
# h integration step (constant)
# t is the time instant just before calling the integration function 
# z0 is the z component of the goal (reference) state
# the vector field in in dvCord(c)
  
# 1st call
    xd = dvCord(x0, e, ep, Ki, Kp, t, a, b, F, G)
    savex0 = x0.reshape((-1,1))  
    phi = xd
    x0 = savex0+np.array(0.5*h*xd)

    
# 2nd call
    xd = dvCord(x0, e, ep, Ki, Kp, t+0.5*h, a, b, F, G)
    phi = phi+2*xd
    x0 = savex0+0.5*h*xd

# 3rd call
    xd = dvCord(x0, e, ep, Ki, Kp, t+0.5*h, a, b, F, G)
    phi = phi+2*xd
    x0 = savex0+h*xd

# 4th call
    xd = dvCord(x0, e, ep, Ki, Kp, t+h, a, b, F, G)
    x = savex0+(phi+xd)*h/6

    return (x)



In [4]:
def IC_Target(w):
# 
# returns randomly selected initial condition and target states.
# w is a two-column array with as many rows as state variables.
# the first column of w has the width of the uniform distribution
# the second column of w has the lower limit of the uniform distribution

    # global ref
    ref = np.array(np.zeros((3,1)))
    for k in range(0, 3):
        # random initial condition
        x0[k+3] = np.random.rand(1,1)*w[k,0]+w[k,1]
        # random reference (target)
        ref[k] = np.random.rand(1,1)*w[k,0]+w[k,1]
        
    return (ref, x0)    

In [5]:
def get_times(e, tol):
#
# returns in t_tg the time index to steady-state for x, y and z, respectively
# this is computed within a tolerance tol for the MCth Monte Carlo run
# to get time, the time index should be multiplied by h

    
    # vector of steady-state times
    t_tg = np.array(np.zeros((3,1)), dtype='int64')

    for i in range(0, 3):
        ki = 0
        while np.abs(e[i,ki]) > tol[i] :
            ki = ki + 1
        t_tg[i] = ki
        
    t_1 = t_tg[0]
    t_2 = t_tg[1]
    t_3 = t_tg[2]
    
    return t_1, t_2, t_3

In [6]:
def get_energy(x, ref, t_x, t_y, t_z, MC, h):
#
# computes the energy spent in taking the state from x0 to ref
# the first three rows of x are the control actions
# t_x is the time to steady-state for x, same for t_y and t_z
# this is computed for the MCth Monte Carlo run

    # consider the longest time for computation purposes
    t_m = max(t_x, t_y, t_z)
    Euc_norm = x[3:6,t_m]-x[3:6,0]
    # Euclidean distance between initial condition and target
    Euc_norm = np.sqrt(sum(Euc_norm**2))
    
    # Normalization is done with respect to the total distance
    E[0,MC] = sum(x[0,0:t_x]**2)*h/Euc_norm
    E[1,MC] = sum(x[1,0:t_y]**2)*h/Euc_norm
    E[2,MC] = sum(x[2,0:t_z]**2)*h/Euc_norm
  
    return E, Euc_norm

In [7]:
def get_energy2(x, ref, t_x, t_y, t_z, MC, h, a, b, F, G):
#
# Like get_energy but here the effect of the vector 
# field is subtracted and the time average is taken (hence the power is computed)

    # consider the longest time for computation purposes
    t_m = max(t_x, t_y, t_z)
    
    # part of the control action is used just to counteract the vector field.
    # This part is subtracted from the control action. Only the part that is
    # effectively used to "move the state" having locally cancelled the vector field
    # is used. In get_energy we use the whole control action
    dif = np.array(np.zeros((3,t_m)))
    dif[0,0:t_m] = x[0,0:t_m] - (-x[4,0:t_m] - x[5,0:t_m] - a*x[3,0:t_m] + a*F)
    dif[1,0:t_m] = x[1,0:t_m] - (x[3,0:t_m]*x[4,0:t_m] - b*x[3,0:t_m]*x[5,0:t_m] -x[4,0:t_m] + G)
    dif[2,0:t_m] = x[2,0:t_m] - (b*x[3,0:t_m]*x[4,0:t_m] + x[3,0:t_m]*x[5,0:t_m] - x[5,0:t_m])
    
    # Normalization is done with respect to the total time
    E2[0,MC] = sum(dif[0,0:t_x]**2)*h/t_m
    E2[1,MC] = sum(dif[1,0:t_y]**2)*h/t_m
    E2[2,MC] = sum(dif[2,0:t_z]**2)*h/t_m
  
    return E2

In [8]:
def get_max_energy(E):
#
    # get the maximum energy per variable over all MC Monte Carlo realizations
    a = np.max(E,1)
 
    # normalized percentage average energies    
    b = np.mean(E,1)
    b = b*100/np.sum(b)
    c = np.std(E,1)*100/np.sum(b) 
              
    return a, b, c

In [9]:
def get_tot_energy(E):
#
    # get average total (adding three variables) energy for each MC Monte Carlo realization
    a = np.sum(E,0)
 
    # total energy averaged over the Monte Carlo realizations, and respective std 
    b = np.mean(a)
    c = np.std(a)
    d = np.median(a)
              
    return b, c, d

In [13]:
# main block

# integration interval
h = 0.01;
t0 = 0;

# final time
tf = 200/h;
t = np.arange(0.0, tf+1, 1)*h
points = np.linspace(1,len(t)-1,len(t)-1, dtype='int64')

# standard parameters
a = 0.258
b = 4.033 
F = 8
G = 1
    
# controller gains (not optimized)
# balanced scheme
# Kp = [20, 20, 20]
# Ki = [2, 2, 2]


###############
# emphasized scheme
# emphasized variable x
# Kp = [100, 10, 10]
# Ki = [10, 2, 2]

# emphasized variable y
# Kp = [10, 100, 10]
# Ki = [2, 10, 2]

# emphasized variable z
# Kp = [10, 10, 100]
# Ki = [2, 2, 10]

##############
# depreciating scheme
# depreciated variable x
# Kp = [4.5, 45, 45]
# Ki = [0.3, 3, 3]

# depreciated variable y
# Kp = [45, 4.5, 45]
# Ki = [3, 0.3, 3]

# depreciated variable: z
Kp = [45, 45, 4.5]
Ki = [3, 3, 0.3]

# the initial error derivative is null
ep = [0, 0, 0]

t_x = 0
t_y = 0
t_z = 0

# number of Monte Carlo runs 
MC = 500

# random inicial conditions for the system and zero inicial conditions for the controllers
# np.random.seed(0)
x0 = np.array(np.zeros((6,1)))

# random reference (target)
ref = np.array(np.zeros((3,1)))

# vector of tolerances
tol = np.array(np.zeros((3,1)))

# matrix with energies and normalized averaged energies
E = np.array(np.zeros((3,MC)))
E2 = np.array(np.zeros((3,MC)))
Euc_norm = np.array(np.zeros((1,MC)))
ena = np.array(np.zeros((3,1)))
Ena = np.array(np.zeros((3,1)))
Ena2 = np.array(np.zeros((3,1)))

ta = np.array(np.zeros((3,1)))
ts = np.array(np.zeros((3,1)))

# vector with maximum energies
em = np.array(np.zeros((3,1)))
Em = np.array(np.zeros((3,1)))
Em2 = np.array(np.zeros((3,1)))

# counter of situations that converge to target
conv = 0


# defines the ranges in R^3 to be investigated, used in IC_Target to 
# randomly select initial conditions and target states
w = np.array(np.zeros((3,2)))
#
# x range: (-7, 7)
w[0, 0] = 14
w[0, 1] = -7
tol[0] = w[0, 0]/100

# y range: (-20,35)
w[1, 0] = 55
w[1, 1] = -20
tol[1] = w[1, 0]/100

# z range: (-25, 35)
w[2, 0] = 60
w[2, 1] = -25
tol[2] = w[2, 0]/100


# Monte Carlo Loop
for j in range(0, MC):

    print(j)
    
    # select random initial condition and target
    ref, x0 = IC_Target(w)

    # this array will contain the trajectory. Row 3 is variable x, row 4 is y, row 5 is z.
    # each column corresponds to an integration step. This "trajectory" array is initialized
    # with zeros
    x = np.array(np.zeros((len(x0),len(t))))

    # place the initial condition as the first column of the array
    x[:,0] = x0.reshape((1,-1)) 

    # initial error
    e = np.array(np.zeros((3,len(t))))
    e[0,0] = ref[0] - x0[3]
    e[1,0] = ref[1] - x0[4]
    e[2,0] = ref[2] - x0[5]

    # calls the Runge Kutta integration scheme
    for k in points:
        # error "derivative" for x, y and z variables (current - previous error)
        ep[0] = (ref[0] - x[3,k-1] - e[0,k-1])/h
        ep[1] = (ref[1] - x[4,k-1] - e[1,k-1])/h
        ep[2] = (ref[2] - x[5,k-1] - e[2,k-1])/h
        
        # error for x, y and z variables
        e[0,k] = ref[0] - x[3,k-1]
        e[1,k] = ref[1] - x[4,k-1]
        e[2,k] = ref[2] - x[5,k-1]
    
        x[:,[k]] = rkCord(x[:,k-1], e[:,k], ep, Ki, Kp, h, t[k], a, b, F, G)
        
    
    
    # if convergence happened
    if max(np.abs(e[:,len(t)-1])) < min(tol) :
        t_x, t_y, t_z = get_times(e, tol)
        E, E_n = get_energy(x, ref, t_x[0], t_y[0], t_z[0], conv, h) 
        E2 = get_energy2(x, ref, t_x[0], t_y[0], t_z[0], conv, h, a, b, F, G) 
        Euc_norm[0,conv] = E_n
        conv = conv + 1

print('ok')
Em, Ena, Esa = get_max_energy(E[:,0:conv-1])   
# print(Em)
print(Ena)
print(Esa)
Em2, Ena2, Esa2 = get_max_energy(E2[:,0:conv-1])
# print(Em2)
print(Ena2)
print(Esa2)
print(conv)

Eat, Est, Emt = get_tot_energy(E[:,0:conv-1])   
# average total energy
print(Eat)
# std total energy
print(Est)
# median total energy
print(Emt)
Eat2, Est2, Emt2 = get_tot_energy(E2[:,0:conv-1])   
# average total energy
print(Eat2)
# std total energy
print(Est2)
# median total energy
print(Emt2)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27